### Excercise 2 Second: Write a query that directly answers a predetermined question from a business stakeholder

In [1]:
#!conda install pandasql

In [2]:
#importing libraries
import pandas as pd
import numpy as np
from pandasql import sqldf

In [3]:
#read json lines using pands
brands=pd.read_json("C:\\Users\\georg\\Downloads\\Brands\\brands.json",lines=True)
receipts=pd.read_json("C:\\Users\\georg\\Downloads\\Receipts\\receipts.json",lines=True)
users=pd.read_json("C:\\Users\\georg\\Downloads\\Users\\users.json",lines=True)

In [4]:
brands_copy=brands.copy()
#clean data
brands_copy['_id'] = brands_copy['_id'].map(lambda x: x['$oid'])
brands_copy['cpg']=brands_copy['cpg'].map(lambda x: x['$id']['$oid'])
#removed cpg_ref since its not relevant for the exercise
brands_copy.tail()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
1162,5f77274dbe37ce6b592e90c0,511111116752,Baking,BAKING,5f77274dbe37ce6b592e90bf,test brand @1601644365844,NaN,NaN
1163,5dc1fca91dda2c0ad7da64ae,511111706328,Breakfast & Cereal,NaN,53e10d6368abd3c7065097cc,Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL
1164,5f494c6e04db711dd8fe87e7,511111416173,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7,test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217
1165,5a021611e4b00efe02b02a57,511111400608,Grocery,NaN,5332f5f6e4b03c9a25efd0b4,LIPTON TEA Leaves,0.0,LIPTON TEA Leaves
1166,6026d757be37ce6369301468,511111019930,Baking,BAKING,6026d757be37ce6369301467,test brand @1613158231643,0.0,TEST BRANDCODE @1613158231644


In [5]:
receipts_copy=receipts.copy()
brands_receipts=pd.DataFrame()
#clean data from $date
receipts_copy['_id'] = receipts_copy['_id'].map(lambda x: x['$oid'])
receipts_copy['dateScanned'] = receipts_copy['dateScanned'].map(lambda x: x['$date'])
#rewardsreceiptsitemlist to be exploded
brands_receipts[['dateScanned','rewardsReceiptItemList','rewardsReceiptStatus','totalSpent']] = receipts_copy[['dateScanned','rewardsReceiptItemList','rewardsReceiptStatus','totalSpent']]
#these are important columns necessity  
brands_receipts.head()

,dateScanned,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent
0,1609687531000,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0
1,1609687483000,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0
2,1609687537000,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0
3,1609687534000,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0
4,1609687506000,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0


In [6]:
users_copy=users.copy()
#clean data
users_copy['_id'] = users_copy['_id'].map(lambda x: x['$oid'])
users_copy['createdDate'] = users_copy['createdDate'].map(lambda x: x['$date'])
users_copy['lastLogin'] = users_copy['lastLogin'].map(lambda x: x['$date'],na_action='ignore')
users_copy.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,1609687530554,1.609688e+12,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI


In [7]:
#creating an sql query to avoid passing globals everytime
pysqldf = lambda q: sqldf(q, globals())

### When considering total number of item purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [8]:
status_spent=brands_receipts[['rewardsReceiptStatus','totalSpent']]
query="""SELECT CASE
    WHEN rewardsReceiptStatus='FINISHED' THEN 'Accepted' --Finished is Accepted
    WHEN rewardsReceiptStatus='FLAGGED' THEN 'Other'    --Flagged might not be Rejected
    WHEN rewardsReceiptStatus='REJECTED' THEN 'Rejected'
    WHEN rewardsReceiptStatus='PENDING' THEN 'Other'
    WHEN rewardsReceiptStatus='SUBMITTED' THEN 'Accepted'
    END AS statusReceipt,AVG(totalSpent) as averageSpent
                FROM status_spent
                WHERE statusReceipt!='Other' --if we you want to check flagged and pending just remove this
                GROUP BY statusReceipt"""

In [9]:
pysqldf(query)

,statusReceipt,averageSpent
0,Accepted,80.854305
1,Rejected,23.326056


#### Average Total Spent on Accepted is greater than Rejected

### T

### Exercise 3 Third: Evaluate Data Quality Issues in the Data Provided

In [10]:
brands_receipts.head()

,dateScanned,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent
0,1609687531000,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0
1,1609687483000,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0
2,1609687537000,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0
3,1609687534000,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0
4,1609687506000,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0


In [12]:
New_brands_receipts=brands_receipts.explode('rewardsReceiptItemList',ignore_index=True)

In [21]:
New_brands_receipts=New_brands_receipts[New_brands_receipts['rewardsReceiptStatus']=="FINISHED"]

In [47]:
New_brands_receipts['rewardsReceiptItemList'].map(lambda x: x['barcode'])

KeyError: 'barcode'

In [46]:
num=0
for i in New_brands_receipts['rewardsReceiptItemList']:
    try:
        if i['barcode']==True:
            print("Working")
    except KeyError:
        print(i)
print(num)

{'barcode': '4011', 'description': 'ITEM NOT FOUND', 'finalPrice': '26.00', 'itemPrice': '26.00', 'needsFetchReview': False, 'partnerItemId': '1', 'preventTargetGapPoints': True, 'quantityPurchased': 5, 'userFlaggedBarcode': '4011', 'userFlaggedNewItem': True, 'userFlaggedPrice': '26.00', 'userFlaggedQuantity': 5}
{'barcode': '4011', 'description': 'ITEM NOT FOUND', 'finalPrice': '1', 'itemPrice': '1', 'partnerItemId': '1', 'quantityPurchased': 1}
{'barcode': '028400642255', 'description': 'DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCED FAT BAG 1 OZ', 'finalPrice': '10.00', 'itemPrice': '10.00', 'needsFetchReview': True, 'needsFetchReviewReason': 'USER_FLAGGED', 'partnerItemId': '2', 'pointsNotAwardedReason': 'Action not allowed for user and CPG', 'pointsPayerId': '5332f5fbe4b03c9a25efd0ba', 'preventTargetGapPoints': True, 'quantityPurchased': 1, 'rewardsGroup': 'DORITOS SPICY SWEET CHILI SINGLE SERVE', 'rewardsProductPartnerId': '5332f5fbe4b03c9a25efd0ba', 'userFlaggedBarcode': '0284

TypeError: 'float' object is not subscriptable

### One Data Quality Issues 
In the column "rewardsReceiptItemList" the barcode and brandcode are present alternatively its better to have one standardized key for the items in list. choosing between barcode and brandcode could be difficult